In [6]:
# THIS FILE SHOULD BE OPENED IN GOOGLE COLAB

# ABOUT THE ALGORITHM --------------------------------------------------------
# The center/restaurant/scoring/search/etc algorithm in CenterPlate currently
# as of 6/6/25

# 1. Gets the midpoint of all users (point 1)

# 2. Gets a few populated cities close to the centerpoint (points 2, 3, 4, etc)
#     2.5. There's a github file we use that has the xx,000 most populated U.S. cities
#          and their coordinates/population/etc.

# 3. Calculates the AVERAGE SCORE of the TOP ~20 restaurants FOR EACH POINT

# 4. (score = how well does the restaurant fit the filters, minus a distance penalty)
#     4.5. We see how well the restaurants fit the filters by
#          just running a new search for every combination of filters.
#          If CenterPlate ever runs out of the free tier of FourSquare's
#          API, you'll need to score restaurants based on their tags in
#          the FourSquare API JSON response.
#          Small beta tests and debugging alone will never use more than 5-10% of the
#          monthly free credits of the FourSquare API.

# 5. The point with the highest AVERAGE RESTAURANT SCORE (among the top 20) is chosen.

# The code below is a demo of JUST the scoring system.



# GO GET A FOURSQUARE API KEY FROM FOURSQUARE AND PUT IT HERE:
FOURSQUARE_API_KEY = "GetAnApiKeyFromFoursquare"
if FOURSQUARE_API_KEY == "GetAnApiKeyFromFoursquare":
  print("Get an API key from FourSquare and plug it in at the top of the code")

################################################################################

import requests
import math
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="ctierne1@stevens.edu") #openstreetmaps

def geographic_midpoint(locations):
    """
    Finds the geographic midpoint of a list of latitude, longitude pairs.

    The approach taken is to convert each location to cartesian coordinates
    (x, y, z), average them, and then convert the result back to latitude and
    longitude.

    Parameters
    ----------
    locations : list of (float, float)
        A list of (latitude, longitude) pairs.

    Returns
    -------
    tuple of two floats
        The geographic midpoint of the input locations as a (latitude, longitude)
        pair.
    """
    x=y=z=0.0
    for lat, lon in locations:
        lat, lon = math.radians(lat), math.radians(lon)
        x += math.cos(lat) * math.cos(lon)
        y += math.cos(lat) * math.sin(lon)
        z += math.sin(lat)

    # Average the coordinates
    x /= len(locations)
    y /= len(locations)
    z /= len(locations)

    # Convert cartesian coordinates back to latitude and longitude
    lon = math.atan2(y, x)
    hyp = math.sqrt(x * x + y * y)
    lat = math.atan2(z, hyp)

    # Convert the result back to degrees
    return math.degrees(lat), math.degrees(lon)


def find_nearest_city(latitude, longitude):
    """
    Finds the nearest city to a given latitude and longitude.

    Parameters
    ----------
    latitude : float
        The latitude of the point to find the nearest city to.
    longitude : float
        The longitude of the point to find the nearest city to.

    Returns
    -------
    str
        The name of the nearest city to the given latitude and longitude. If
        no city is found, returns "City not found".
    """
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    if location:
        return location.address
    return "City not found"


def get_coordinates_from_city(city_name):
    """
    Retrieves the latitude and longitude of a given city name.

    Parameters
    ----------
    city_name : str
        The name of the city for which to retrieve coordinates.

    Returns
    -------
    tuple of (float, float) or (None, None)
        A tuple containing the latitude and longitude of the city if found,
        otherwise (None, None).
    """
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    print("City not found. Please check the spelling or try another city.")
    return None, None

def get_locations():
    """
    Retrieves the latitude and longitude of a list of cities.

    Parameters
    ----------
    None

    Returns
    -------
    list of tuples of (float, float)
        A list of tuples containing the latitude and longitude of the cities.
        If a city is not found, it is skipped and a message is printed.
    """
    city_names = input("Enter city names separated by commas: ")
    city_list = [city.strip() for city in city_names.split(",")]  # Split and strip spaces
    locations = []
    for city in city_list:
        lat, lon = get_coordinates_from_city(city)
        if lat is not None and lon is not None:
            locations.append((lat, lon))
        else:
            print(f"Could not retrieve coordinates for {city}.")
    return locations



### foursquare initialization
import folium
HEADERS = {"Authorization": FOURSQUARE_API_KEY}
BASE_URL = "https://api.foursquare.com/v3/places/search"


# Display map with locations and centerpoint
def display_map(locations, center_lat, center_lon):
    """
    Displays a map with markers for each location and a center point.

    This function creates a folium map centered at the specified latitude and longitude.
    It places a blue marker for each location in the provided list and a red marker
    for the center of minimum distance.

    Parameters
    ----------
    locations : list of tuples of (float, float)
        A list of tuples where each tuple contains the latitude and longitude of a location.
    center_lat : float
        The latitude for the center point marker.
    center_lon : float
        The longitude for the center point marker.

    Returns
    -------
    folium.Map
        A folium map object with the added markers.
    """
    midpoint_map = folium.Map(location=[center_lat, center_lon], zoom_start=6, width='600px', height='400px')
    for i, (lat, lon) in enumerate(locations):
        folium.Marker([lat, lon], popup=f"Location {i + 1}", icon=folium.Icon(color="blue")).add_to(midpoint_map)
    folium.Marker([center_lat, center_lon], popup="Center of Minimum Distance", icon=folium.Icon(color="red")).add_to(midpoint_map)
    return midpoint_map


def search_restaurants_combined(latitude, longitude, price_level=None, dietary_restriction=None, cuisine=None, parking=None):
    """
    Searches for restaurants around a given location, with optional filters for price level, dietary restrictions, cuisine, and parking availability.

    Parameters
    ----------
    latitude : float
        The latitude of the location to search around.
    longitude : float
        The longitude of the location to search around.
    price_level : str or None, optional
        The price level to search for. If None, no filter is applied. Otherwise, should be one of "1", "2", "3", or "4", in order of increasing price.
    dietary_restriction : str or None, optional
        The dietary restriction to search for. If None, no filter is applied. Otherwise, the search query will be filtered to include only restaurants with the given dietary restriction.
    cuisine : str or None, optional
        The cuisine to search for. If None, no filter is applied. Otherwise, the search query will be filtered to include only restaurants with the given cuisine.
    parking : bool or None, optional
        Whether to filter by parking availability. If None, no filter is applied. If True, the search query will be filtered to include only restaurants with parking. If False, the search query will be filtered to include only restaurants with public transport.

    Returns
    -------
    dict
        A dictionary with the response from the Foursquare API. The structure of the response is documented at https://developer.foursquare.com/docs/api/venues/search.
    """
    params = {
        "ll": f"{latitude},{longitude}",
        "categories": "13065",  # Restaurant category
        "limit": 20  # Adjust the number of results if needed
    }

    # Add filters if provided
    if price_level:
        params["price"] = price_level
    if dietary_restriction:
        params["query"] = dietary_restriction
    if cuisine:
        params["query"] = f"{params.get('query', '')} {cuisine}".strip()  # Combine query terms
    if parking is not None:
        keyword = "parking" if parking else "public transport"
        params["query"] = f"{params.get('query', '')} {keyword}".strip()

    # Make the API request
    response = requests.get(BASE_URL, headers=HEADERS, params=params)
    return response.json()


from itertools import combinations

def search_restaurants_combined_scored(latitude, longitude, filters):
    # List to store restaurant matches with scores
    """
    Search for restaurants given a list of filters and return a sorted list of restaurants
    with scores. The score is the number of filter combinations that the restaurant matches.

    Parameters
    ----------
    latitude : float
        The latitude of the search location.
    longitude : float
        The longitude of the search location.
    filters : list of str
        A list of filters to apply to the search. Each filter is a string that is
        combined with the other filters using the AND operator.

    Returns
    -------
    list of dict
        A list of dictionaries, each representing a restaurant with a score.
        The dictionary contains the keys "name", "address", and "score". The
        "score" is the number of filter combinations that the restaurant matches.
    """
    restaurant_scores = {}

    # Generate all combinations of filters
    filter_combinations = []
    for r in range(1, len(filters) + 1):
        filter_combinations.extend(combinations(filters, r))

    for combo in filter_combinations:
        # Prepare query string by combining filters
        query = " ".join(combo)
        params = {
            "ll": f"{latitude},{longitude}",
            "categories": "13065",
            "query": query,
            "limit": 10
        }

        # Make the API request
        response = requests.get(BASE_URL, headers=HEADERS, params=params)
        results = response.json().get("results", [])

        # Update restaurant scores
        for restaurant in results:
            restaurant_id = restaurant.get("fsq_id")
            if restaurant_id not in restaurant_scores:
                restaurant_scores[restaurant_id] = {
                    "name": restaurant.get("name"),
                    "address": restaurant.get("location", {}).get("address", "Unknown address"),
                    "score": 0,
                    "distance": 0

                }
            # Calculate the distance using the Haversine formula
            lat1, lon1 = latitude, longitude
            lat2, lon2 = restaurant.get("geocodes", {}).get("main", {}).get("latitude"), restaurant.get("geocodes", {}).get("main", {}).get("longitude")
            # Radius of the Earth in kilometers
            radius = 6371
            # Calculate the difference in latitude and longitude
            dlat = math.radians(lat2 - lat1)
            dlon = math.radians(lon2 - lon1)
            a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            distance = radius * c

            # Scale the score based on the distance
            # Calculate score based on distance
            # The scaling factor determines how much the distance influences the score.
            # A smaller scaling factor will result in a higher score for closer distances.
            # The distance is measured in kilometers.
            scaling_factor = 500  # Adjust this value to change the influence of distance on the score
            score = 1 / (1 + distance / scaling_factor)

            restaurant_scores[restaurant_id]["score"] += score
            restaurant_scores[restaurant_id]["distance"] = distance


    # Sort restaurants by score
    sorted_restaurants = sorted(restaurant_scores.values(), key=lambda x: x["score"], reverse=True)
    return sorted_restaurants

#calling the midpoint function
locations = get_locations()
center_lat, center_lon = geographic_midpoint(locations)
print(f"The center of minimum distance is located at: Latitude = {center_lat}, Longitude = {center_lon}")
nearest_city = find_nearest_city(center_lat, center_lon)
print(f"The nearest city to this midpoint is: {nearest_city}")


#getting user inputs for preferences
price_level = ""
dietary_restriction = ""
cuisine = ""
parking_query = ""
price_level = input("Enter price level (1-4, or leave blank for no filter): ")
dietary_restriction = input("Enter dietary restriction (e.g., vegan, or leave blank for no filter): ")
cuisine = input("Enter cuisine (e.g., Italian, or leave blank for no filter): ")
parking_query = input(f"Enter \"parking\" or leave blank for no filter: ")
parking_query += input(f"Enter \"public transport\" or leave blank for no filter: ")

#calling function to get search results
results = search_restaurants_combined_scored(center_lat, center_lon, [price_level, dietary_restriction, cuisine, parking_query])

#displaying search results
if results:
    max_score = max(restaurant.get("score", 0) for restaurant in results)
    print("\nFiltered Restaurants:")
    for restaurant in results:
        name = restaurant.get("name", "Unknown")
        address = restaurant.get("address", "Unknown address")
        score = restaurant.get("score", 0)
        distance = restaurant.get("distance", 0)
        percentage_score = (score / max_score) * 100
        print(f"- {name} at {address} (Score: {percentage_score:.2f}%, Distance: {distance:.2f} km)")
else:
    print("No results found.")


#display map
map_display = display_map(locations, center_lat, center_lon)
map_display.save("midpoint_map.html")
map_display


Get an API key from FourSquare and plug it in at the top of the code


KeyboardInterrupt: Interrupted by user